# Section 05 — Hybrid ML Using Pretrained CNN Features

This section uses the best pretrained CNN (ResNet50) as a deep feature extractor.

We remove the final classification layer and extract a 2048-dimensional feature vector for each CT scan.  
These features are used to train classical machine learning models:

- Logistic Regression
- KNN
- Random Forest

Hybrid models often train faster and allow easier interpretability compared to end-to-end deep learning.

Feature extraction is performed for train, validation, and test sets.  
Each ML model is trained on extracted features and evaluated using the same metrics as the CNN models.

In [7]:
# import necessary libraries
import torch
import torch.nn as nn
import numpy as np

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import pickle
import os

os.makedirs("models", exist_ok=True)

In [8]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

# Train data loaders with augmentations
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Validation data loaders without augmentations
val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
val_dataset = datasets.ImageFolder("data/validation",   transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)

In [9]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    # Define forward pass
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
# Load the best saved model
best_model = CNNModel(num_classes=4)
best_model.load_state_dict(torch.load("models/cnn_model.pth", weights_only=True))
best_model.eval()

CNNModel(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)

In [10]:
# Feature extractor using the trained CNN
class CNNFeatureExtractor(nn.Module):
    def __init__(self, trained_model):
        super().__init__()
        self.conv1 = trained_model.conv1
        self.conv2 = trained_model.conv2
        self.conv3 = trained_model.conv3
        self.pool  = trained_model.pool
        self.fc1   = trained_model.fc1

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        return x
    
def extract_features(model, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()

    all_X, all_y = [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            feats = model(images)      # (B, 512)
            all_X.append(feats.cpu().numpy())
            all_y.append(labels.numpy())

    return np.vstack(all_X), np.hstack(all_y)

In [11]:
# Train hybrid models
def train_hybrid(best_model, train_loader, val_loader):
    extractor = CNNFeatureExtractor(best_model)

    print("Extracting CNN feature vectors...")
    X_train, y_train = extract_features(extractor, train_loader)
    X_val,   y_val   = extract_features(extractor, val_loader)

    results = {}

    # random forest
    rf = RandomForestClassifier(n_estimators=300, random_state=42)
    rf.fit(X_train, y_train)

    val_acc = accuracy_score(y_val, rf.predict(X_val))
    print("\n[RF] Validation Accuracy:", val_acc)
    results["RF"] = val_acc

    with open("models/rf_hybrid.pkl", "wb") as f:
        pickle.dump(rf, f)

    # logistic regression
    scaler_lr = StandardScaler()
    X_train_lr = scaler_lr.fit_transform(X_train)
    X_val_lr   = scaler_lr.transform(X_val)

    lr = LogisticRegression(max_iter=2000)
    lr.fit(X_train_lr, y_train)

    val_acc = accuracy_score(y_val, lr.predict(X_val_lr))
    print("[LR] Validation Accuracy:", val_acc)
    results["LR"] = val_acc

    with open("models/lr_hybrid.pkl", "wb") as f:
        pickle.dump(lr, f)
    with open("models/lr_scaler.pkl", "wb") as f:
        pickle.dump(scaler_lr, f)

    # k-nearest neighbors
    scaler_knn = StandardScaler()
    X_train_knn = scaler_knn.fit_transform(X_train)
    X_val_knn   = scaler_knn.transform(X_val)

    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_knn, y_train)

    val_acc = accuracy_score(y_val, knn.predict(X_val_knn))
    print("[KNN] Validation Accuracy:", val_acc)
    results["KNN"] = val_acc

    with open("models/knn_hybrid.pkl", "wb") as f:
        pickle.dump(knn, f)
    with open("models/knn_scaler.pkl", "wb") as f:
        pickle.dump(scaler_knn, f)

    return results

In [12]:
hybrid_results = train_hybrid(
    best_model,
    train_loader=train_loader,
    val_loader=val_loader
)

print("\nHybrid Validation Accuracies:")
print(hybrid_results)

Extracting CNN feature vectors...

[RF] Validation Accuracy: 0.888503937007874
[LR] Validation Accuracy: 0.8869291338582678
[KNN] Validation Accuracy: 0.8692913385826772

Hybrid Validation Accuracies:
{'RF': 0.888503937007874, 'LR': 0.8869291338582678, 'KNN': 0.8692913385826772}
